In [1]:
from utils import plot_label_freq, CNNBackbone, ASTBackbone, LSTMBackbone, SpectrogramDataset, CLASS_MAPPING, get_device, train, torch_train_val_split, set_seed, test_model, plot_train_val_losses, get_classification_report
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report
import numpy as np

In [2]:
BATCH_SIZE = 8
DATA_PATH = "/home/alex/Downloads/archive(1)/data/"
RANDOM_SEED = 42
DEVICE = get_device()
NUM_CATEGORIES = 10
cnn_in_channels = 1
cnn_filters = [32, 64]
cnn_out_feature_size = 64

In [3]:
class Classifier(nn.Module):
    def __init__(self, num_classes, backbone):
        """
        backbone (nn.Module): The nn.Module to use for spectrogram parsing
        num_classes (int): The number of classes
        """
        super(Classifier, self).__init__()
        self.backbone = backbone  # An LSTMBackbone or CNNBackbone
        self.is_lstm = isinstance(self.backbone, LSTMBackbone)
        self.output_layer = nn.Linear(self.backbone.feature_size, num_classes)
        self.criterion = nn.CrossEntropyLoss() if num_classes > 1 else nn.MSELoss()

    def forward(self, x, targets, lengths):
        if not self.is_lstm:
            feats = self.backbone(x)  
        else:
            self.backbone(x, lengths)
        logits = self.output_layer(feats)
        logits = logits.squeeze(-1) if self.criterion.__class__ == nn.modules.loss.MSELoss else logits
        loss = self.criterion(logits, targets)
        return loss, logits, feats

In [4]:
mel_specs_test_data = SpectrogramDataset(DATA_PATH + "fma_genre_spectrograms/", class_mapping=CLASS_MAPPING, train=False)
mel_specs_test_dl, _ = torch_train_val_split(dataset=mel_specs_test_data, batch_eval=BATCH_SIZE, batch_train=BATCH_SIZE, val_size=.0, shuffle=False)

In [5]:
backbone = CNNBackbone(input_dims=mel_specs_test_data.feats[0].shape, in_channels=cnn_in_channels, filters=cnn_filters, feature_size=cnn_out_feature_size)

IndexError: list index out of range

In [ ]:
backbone = CNNBackbone(input_dims=mel_specs_test_data.feats[0].shape, in_channels=cnn_in_channels, filters=cnn_filters, feature_size=cnn_out_feature_size)  
model = Classifier(num_classes=NUM_CATEGORIES, backbone=backbone).to(device=DEVICE)
model.load_state_dict(torch.load("weight_path", weights_only=True))


FileNotFoundError: [Errno 2] No such file or directory: 'weight_path'

In [6]:
set_seed(RANDOM_SEED)
backbone = CNNBackbone(input_dims=mel_specs_test_data.feats[0].shape, in_channels=cnn_in_channels, filters=cnn_filters, feature_size=cnn_out_feature_size)
set_seed(RANDOM_SEED)  
model = Classifier(num_classes=NUM_CATEGORIES, backbone=backbone).to(device=DEVICE)


IndexError: list index out of range

In [7]:
def extract_latents(model, test_dl, device):
    model.to(device)
    features = []
    for inputs, targets, lenghts in test_dl:
        inputs, targets, lenghts = inputs.to(device), targets.to(device), lenghts.to(device)
        print(inputs.device)
        _, _, feats = model(inputs, targets, lenghts)
        features.append(feats)

    return features


In [8]:
features = extract_latents(model, mel_specs_test_dl, DEVICE)

NameError: name 'model' is not defined